In [1]:
import cv2
import torch
from ultralytics import YOLO

In [2]:
def detect_coke_bottle(image):
    """Detects Coca-Cola bottles and returns their bounding boxes and pixel heights."""
    results = coke_model(image)
    coke_boxes = []

    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Get bounding box coordinates
            height_pixels = y2 - y1  # Calculate height in pixels
            coke_boxes.append((x1, y1, x2, y2, height_pixels))

    return coke_boxes

In [3]:
def detect_height_bottle(image_path, model):
    """Detect Coca-Cola bottles and return their bounding box height in pixels."""
    image = cv2.imread(image_path)
    results = model(image)
    
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = box.xyxy[0]  # Bounding box coordinates
            height_pixels = y2 - y1  # Bottle height in pixels
            real_height_cm = 26  # Known height of Coca-Cola bottle
            
            pixel_to_cm_ratio = real_height_cm / height_pixels
            estimated_real_height = height_pixels * pixel_to_cm_ratio
            
            return height_pixels, estimated_real_height

    return None, None  # If no bottle is detected

In [4]:
import torch
import cv2
import numpy as np
from ultralytics import YOLO

In [5]:
def load_model(model_path):
    model = YOLO(model_path)  # Load YOLO model
    return model

In [6]:
def detect_objects(model, image_path):
    image = cv2.imread(image_path)
    results = model(image)  # Perform object detection
    return results, image


In [7]:
def get_ceiling_floor_coordinates(results):
    """Get Y-coordinates of the ceiling and floor."""
    ceiling_y, floor_y = None, None
    
    for result in results:
        for box in result.boxes:
            cls = int(box.cls.cpu().numpy().item())  # Extract class label
            y_min, y_max = int(box.xyxy[0][1]), int(box.xyxy[0][3])
            
            if cls == 0:  # Assuming class 0 is 'ceiling'
                ceiling_y = y_min
            elif cls == 1:  # Assuming class 1 is 'floor'
                floor_y = y_max
    
    return ceiling_y, floor_y

In [8]:
def calculate_height(ceiling_y, floor_y, calibration_obj):
    """Calculate ceiling height using a calibration object (Coca-Cola bottle)."""
    if ceiling_y is None or floor_y is None:
        print("Error: Ceiling or Floor not detected!")
        return None  # Avoid TypeError

    pixel_height = floor_y - ceiling_y

    if calibration_obj:
        real_height, obj_pixel_height = calibration_obj
        pixel_to_cm_ratio = real_height / obj_pixel_height
        return pixel_height * pixel_to_cm_ratio
    
    return pixel_height  # Return pixel height if no calibration object


In [9]:
coke_model_path = "best (8).pt"
ceiling_floor_model_path = "best (5).pt"
coke_model = YOLO(coke_model_path)  # Load Coca-Cola bottle detection model
ceiling_floor_model = YOLO(ceiling_floor_model_path)

In [10]:
def detect_coke_bottle(image):
    """Detects Coca-Cola bottles and returns their bounding boxes and pixel heights."""
    results = coke_model(image)
    coke_boxes = []

    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Get bounding box coordinates
            height_pixels = y2 - y1  # Calculate height in pixels
            coke_boxes.append((x1, y1, x2, y2, height_pixels))

    return coke_boxes

In [11]:
def detect_ceiling_floor(image):
    """Detects the ceiling and floor, returning their y-coordinates."""
    results = ceiling_floor_model(image)
    ceiling_y, floor_y = None, None

    for result in results:
        for box in result.boxes:
            cls = int(box.cls.cpu().numpy().item())  # Extract class ID
            y_min, y_max = int(box.xyxy[0][1]), int(box.xyxy[0][3])

            if cls == 0:  # Assuming class 0 is ceiling
                ceiling_y = y_min
            elif cls == 1:  # Assuming class 1 is floor
                floor_y = y_max

    return ceiling_y, floor_y


In [12]:
def draw_annotations(image, coke_boxes, ceiling_y, floor_y, pixel_to_cm_ratio=None):
    """Draws bounding boxes, vertical line, and labels on the image."""
    annotated_image = image.copy()
    
    # Draw Coca-Cola bounding boxes
    for (x1, y1, x2, y2, height_pixels) in coke_boxes:
        cv2.rectangle(annotated_image, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green box
        cv2.putText(annotated_image, f"{height_pixels}px", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Draw vertical line from floor to ceiling
    if ceiling_y is not None and floor_y is not None:
        mid_x = image.shape[1] // 2  # Center x-coordinate
        cv2.line(annotated_image, (mid_x, ceiling_y), (mid_x, floor_y), (0, 0, 255), 2)  # Red line
        
        # Calculate real-world ceiling height if calibration object exists
        pixel_height = floor_y - ceiling_y
        if pixel_to_cm_ratio:
            real_ceiling_height = pixel_height * pixel_to_cm_ratio
            height_text = f"Height: {real_ceiling_height:.2f} cm"
        else:
            height_text = f"Height: {pixel_height} pixels"

        # Add text label
        cv2.putText(annotated_image, height_text, (mid_x + 10, (ceiling_y + floor_y) // 2),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    return annotated_image

In [13]:
import os

In [14]:
'''# Process all images in test_images folder
image_folder = "image_folder"
output_folder = "output_images"
os.makedirs(output_folder, exist_ok=True)  # Ensure output folder exists
for image_name in os.listdir(image_folder):
    if image_name.lower().endswith(('.jpg', '.jpeg', '.png')):
        image_path = os.path.join(image_folder, image_name)
        model_path = "best (8).pt"  # Path to your trained YOLO model
        height_pixels, estimated_real_height =detect_height_bottle(image_path, model_path)

        print(f"Detected Coca-Cola Bottle Height: {height_pixels:.2f} pixels")
        print(f"Estimated Real-World Height: {estimated_real_height:.2f} cm")
        # image = cv2.imread(image_path)

        # Detect objects
        # coke_boxes = detect_coke_bottle(image
        # calibration_obj = (estimated_real_height, height_pixels)
        # Example usage
        model_path1 = "best (5).pt"  # Path to trained YOLO model
         # Input image path
        calibration_obj = (estimated_real_height, height_pixels)  # Example: (real-world height in cm, pixel height)

        model = load_model(model_path1)
        results, image = detect_objects(model, image_path)
        ceiling_y, floor_y = get_ceiling_floor_coordinates(results)
        pixel_height = floor_y - ceiling_y  # Calculate pixel height
        print(f"Pixel Height: {pixel_height} pixels")  # Output the result

        height = calculate_height(ceiling_y, floor_y, calibration_obj)
        print(f"Ceiling Height: {height} cm" if calibration_obj else f"Ceiling Height: {height} pixels")

        
        image = cv2.imread(image_path)
        

        # Detect objects
        coke_boxes = detect_coke_bottle(image)
        ceiling_y, floor_y = detect_ceiling_floor(image)
        
        # Calibration: Use Coca-Cola bottle to calculate pixel-to-cm ratio
        if coke_boxes:
            real_coke_height_cm = 24
            _, _, _, _, height_pixels = coke_boxes[0]  # First detected bottle
            pixel_to_cm_ratio = real_coke_height_cm / height_pixels
        else:
            pixel_to_cm_ratio = None

        # Draw results
        annotated_image = draw_annotations(image, coke_boxes, ceiling_y, floor_y, pixel_to_cm_ratio)

        # Save annotated image
        output_path = os.path.join(output_folder, f"annotated_{image_name}")
        cv2.imwrite(output_path, annotated_image)

print("Processing complete! Check the 'output_images' folder.")'''

'# Process all images in test_images folder\nimage_folder = "image_folder"\noutput_folder = "output_images"\nos.makedirs(output_folder, exist_ok=True)  # Ensure output folder exists\nfor image_name in os.listdir(image_folder):\n    if image_name.lower().endswith((\'.jpg\', \'.jpeg\', \'.png\')):\n        image_path = os.path.join(image_folder, image_name)\n        model_path = "best (8).pt"  # Path to your trained YOLO model\n        height_pixels, estimated_real_height =detect_height_bottle(image_path, model_path)\n\n        print(f"Detected Coca-Cola Bottle Height: {height_pixels:.2f} pixels")\n        print(f"Estimated Real-World Height: {estimated_real_height:.2f} cm")\n        # image = cv2.imread(image_path)\n\n        # Detect objects\n        # coke_boxes = detect_coke_bottle(image\n        # calibration_obj = (estimated_real_height, height_pixels)\n        # Example usage\n        model_path1 = "best (5).pt"  # Path to trained YOLO model\n         # Input image path\n        c

In [15]:
# Define folders
image_folder = "Input_test_folder(3m approx)"
output_folder = "output_test_folder(ceil_height)"
os.makedirs(output_folder, exist_ok=True)  # Ensure output folder exists

In [16]:
coke_model_path = "best (8).pt"
ceiling_floor_model_path = "best (5).pt"
coke_model = YOLO(coke_model_path)  # Load Coca-Cola bottle detection model
ceiling_floor_model = YOLO(ceiling_floor_model_path)
image_heights = {}  # Declare outside to store all images' heights

for image_name in os.listdir(image_folder):
    if image_name.lower().endswith(('.jpg', '.jpeg', '.png')):
        image_path = os.path.join(image_folder, image_name)

        # Detect Coca-Cola bottle height
        height_pixels, estimated_real_height = detect_height_bottle(image_path, coke_model)

        if height_pixels is None:
            print(f"Skipping {image_name}: No Coca-Cola bottle detected.")
            continue

        print(f"Detected Coca-Cola Bottle Height: {height_pixels:.2f} pixels")
        print(f"Estimated Real-World Height: {estimated_real_height:.2f} cm")

        # Detect ceiling and floor
        results = ceiling_floor_model(image_path)
        ceiling_y, floor_y = get_ceiling_floor_coordinates(results)

        if ceiling_y is None or floor_y is None:
            print(f"Skipping {image_name}: Ceiling or floor not detected.")
            continue

        # Calculate ceiling height using Coca-Cola bottle as a calibration object
        calibration_obj = (estimated_real_height, height_pixels)
        ceiling_height = calculate_height(ceiling_y, floor_y, calibration_obj)

        print(f"Ceiling Height: {ceiling_height:.2f} cm")

        # image_heights = {}

        image_heights[image_name] = ceiling_height  # Dynamically update dictionary
        image = cv2.imread(image_path)
        

        # Detect objects
        coke_boxes = detect_coke_bottle(image)
        ceiling_y, floor_y = detect_ceiling_floor(image)
        
        # Calibration: Use Coca-Cola bottle to calculate pixel-to-cm ratio
        if coke_boxes:
            real_coke_height_cm = 24
            _, _, _, _, height_pixels = coke_boxes[0]  # First detected bottle
            pixel_to_cm_ratio = real_coke_height_cm / height_pixels
        else:
            pixel_to_cm_ratio = None

        # Draw results
        annotated_image = draw_annotations(image, coke_boxes, ceiling_y, floor_y, pixel_to_cm_ratio)

        # calculated_height = compute_height(image_name)  # Replace with your actual function
        # image_heights[image_name] = calculated_height  # Dynamically update dictionary
        # Save annotated image
        output_path = os.path.join(output_folder, f"annotated_{image_name}")
        cv2.imwrite(output_path, annotated_image)

print("Processing complete! Check the 'output_images' folder.")


0: 800x608 1 750ml coke, 369.6ms
Speed: 3.5ms preprocess, 369.6ms inference, 0.9ms postprocess per image at shape (1, 3, 800, 608)
Detected Coca-Cola Bottle Height: 79.81 pixels
Estimated Real-World Height: 26.00 cm

image 1/1 /home/priyansh/Philipee OBADIA  Project/Input_test_folder(3m approx)/image44.jpeg: 640x480 1 ceiling, 2 floors, 4 wallss, 322.1ms
Speed: 2.1ms preprocess, 322.1ms inference, 9.2ms postprocess per image at shape (1, 3, 640, 480)
Ceiling Height: 412.45 cm

0: 800x608 1 750ml coke, 354.5ms
Speed: 5.9ms preprocess, 354.5ms inference, 0.7ms postprocess per image at shape (1, 3, 800, 608)

0: 640x480 1 ceiling, 2 floors, 4 wallss, 324.8ms
Speed: 2.2ms preprocess, 324.8ms inference, 9.3ms postprocess per image at shape (1, 3, 640, 480)

0: 800x608 1 750ml coke, 471.5ms
Speed: 5.0ms preprocess, 471.5ms inference, 0.7ms postprocess per image at shape (1, 3, 800, 608)
Detected Coca-Cola Bottle Height: 79.93 pixels
Estimated Real-World Height: 26.00 cm

image 1/1 /home/pri

In [17]:
import numpy as np

# Predicted height (assumed constant for all images as per your requirement)
actual_height = 340  # cm

# Extract actual heights from the provided results
# image_heights = {
#     "image2.jpeg": 197.32,
#     "image5.jpeg": 210.09,
#     "image4.jpeg": 196.10,
#     "image1.jpeg": 120.91,
# }

# List to store actual heights dynamically
predicted_heights = []
real_heights = []

# Fetch actual heights using a loop
# Convert actual heights to a NumPy array
predicted_heights = np.array(list(image_heights.values()))

# Create an array of the actual height for comparison
real_heights = np.full_like(predicted_heights, actual_height)

# Convert to NumPy arrays for calculations
# actual_heights = np.array(actual_heights)
# predicted_heights = np.full_like(actual_heights, predicted_height)

# Compute evaluation metrics
mae = np.mean(np.abs(real_heights - predicted_heights))  # Mean Absolute Error
mse = np.mean((real_heights - predicted_heights) ** 2)  # Mean Squared Error
rmse = np.sqrt(mse)  # Root Mean Squared Error

# Print results
formatted_heights = ", ".join(f"{h:.2f}" for h in predicted_heights)
print(f"Predicted Heights: [{formatted_heights}]")
print(f"MAE: {mae:.2f} cm")
print(f"MSE: {mse:.2f} cm²")
print(f"RMSE: {rmse:.2f} cm")

Predicted Heights: [412.45, 415.40, 431.77, 399.84, 437.79, 462.10, 409.17, 415.18, 455.41, 345.79, 298.62, 339.87, 429.54, 316.05, 474.73, 423.68, 434.88, 408.20, 449.94, 358.32, 419.99, 286.15, 276.17, 401.54, 322.76, 408.67, 376.82, 456.19, 436.41, 448.13, 415.34, 451.88, 337.45, 343.02, 384.07, 393.57, 357.01, 350.25, 410.32, 429.68, 271.60, 335.06, 480.32, 374.31, 385.33, 422.99]
MAE: 65.35 cm
MSE: 5688.21 cm²
RMSE: 75.42 cm


In [18]:
# Calculate Mean Absolute Percentage Error (MAPE)
mape = np.mean(np.abs((real_heights - predicted_heights) / real_heights)) * 100

# Calculate Accuracy
accuracy = 100 - mape

# Print results
print(f"MAPE: {mape:.2f}%")
print(f"Accuracy: {accuracy:.2f}%") 

MAPE: 19.22%
Accuracy: 80.78%
